# SYDE 556/750 --- Assignment 2
**Student ID: 20823934**

*Note:* Please include your numerical student ID only, do *not* include your name.

*Note:* Refer to the [PDF](https://github.com/celiasmith/syde556-f22/raw/master/assignments/assignment_02/syde556_assignment_02.pdf) for the full instructions (including some hints), this notebook contains abbreviated instructions only. Cells you need to fill out are marked with a "writing hand" symbol. Of course, you can add new cells in between the instructions, but please leave the instructions intact to facilitate marking.

In [ ]:
# Import numpy and matplotlib -- you shouldn't need any other libraries
import numpy as np
import matplotlib.pyplot as plt

# Fix the numpy random seed for reproducible results
np.random.seed(18945)

# Some formating options
%config InlineBackend.figure_formats = ['svg']

# 1. Generating a random input signal

## 1.1 Band-limited white noise

**a) Time-domain signals.** Plot $x(t)$ for three randomly generated signals with $\texttt{limit}$ at $5$, $10$, and $20\,\mathrm{Hz}$. For each of these, $\mathtt{T}=1\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$ and $\mathtt{rms}=0.5$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# Create a function that generates a randomly varying x(t) signal chosen from a band limited white noise distribution
# Return both x(t) and X(ω) --> both its time- and Fourier-domain representation
def generate_signal(T, dt, rms, limit, seed):
    """
    T --> The length of the signal in seconds
    dt --> Time time step in seconds
    rms --> The root mean square power of the signal
    limit --> The maximum frequency of the signal
    seed --> Random seed
    """
    np.random.seed(seed)
    ts = np.arange(0, T, dt)                                # Time points
    fs_hz = np.fft.fftshift(np.fft.fftfreq(len(ts), dt))    # Frequency bins in Hz
    fs_rad = 2 * np.pi * fs_hz                              # Frequency bins in rad/s

    # Generate half of the random signal in the frequency domain
    num_samples = len(fs_rad) // 2
    real = np.random.normal(0, 1, num_samples)
    imag = np.random.normal(0, 1, num_samples)
    half_freq_signal = real + 1j * imag

    # Create the full frequency signal by mirroring the half signal
    # Note: The DC component is 0, so the mean of the time domain signal is 0
    freq_signal = np.concatenate((half_freq_signal, np.array([0]), np.conj(np.flip(half_freq_signal))))

    # Cut freq_signal so it has the same length as fs (in case len(fs) is even)
    freq_signal = freq_signal[:len(fs_rad)]

    # Limit the signal to the desired frequency range
    freq_signal[np.abs(fs_hz) > limit] = 0

    # Turn it back into the time domain
    time_signal = np.fft.ifft(np.fft.ifftshift(freq_signal))

    # Check if the time domain signal is real
    imag_threshold = 1e-10  # Error threshold
    if np.all(np.abs(time_signal.imag) < imag_threshold):
        time_signal = time_signal.real
    else:
        raise ValueError("The time domain signal is not real")

    # Scale the time and frequency signals to the RMS power
    old_rms = np.sqrt(np.mean(time_signal ** 2))
    scaling_factor = rms / old_rms
    time_signal *= scaling_factor
    freq_signal *= scaling_factor

    return ts, fs_rad, time_signal, freq_signal

# Adapting the plotting code from the lecture notes
def plot_signals(ts, fs_rad, time_signal, freq_signal):
    # Calculate bandwidth by finding the highest frequency above the energy threshold
    bandwidth = fs_rad[np.max(np.where(np.abs(freq_signal) > 1e-3))]

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(6.5, 2.25))
    ax1.plot(ts, time_signal)
    ax1.set_xlabel('Time $t$ (s)')
    ax1.set_ylabel('Signal x')
    ax1.set_title('Time domain')

    ax2.plot(fs_rad, np.abs(freq_signal))
    ax2.set_xlabel('Frequency $f$ (rad/s)')
    ax2.set_ylabel('Power spectrum $|X|$')
    ax2.set_title('Frequency domain')

    ax3.plot(fs_rad, np.abs(freq_signal))
    ax3.set_xlabel('Frequency $f$ (rad/s)')
    ax3.set_ylabel('Power spectrum $|X|$')
    ax3.set_xlim(-min(20*2*np.pi, bandwidth), min(20*2*np.pi, bandwidth))
    ax3.set_title('Frequency domain (magnified)')

    fig.tight_layout(pad=0.5)

    plt.show()

# Generate and plot the signals
for limit in [5, 10, 20]:
    ts, fs_rad, time_signal, freq_signal = generate_signal(T=1, dt=0.001, rms=0.5, limit=limit, seed=0)
    print(f'Bandwidth: {limit} Hz = {np.round(limit * 2 * np.pi, 1)} rad/s')
    plot_signals(ts, fs_rad, time_signal, freq_signal)

**b) Average power spectrum.** Plot the average $|X(\omega)|$ (the norm of the Fourier coefficients, or “power spectrum”) over $100$ signals generated with $\mathtt{T}=1\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$, $\mathtt{rms}=0.5$, and $\mathtt{limit}=10\,\mathrm{Hz}$ (of course, each of these 100 signals should have a different `seed`). The plot should have the $x$-axis labeled “$\omega$ in radians” and the average $|X|$ value for that $\omega$ on the $y$-axis.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

sum_norm_coefficients = 0
for i in range(100):
    ts, fs_rad, time_signal, freq_signal = generate_signal(T=1, dt=0.001, rms=0.5, limit=10, seed=i)
    sum_norm_coefficients += np.abs(freq_signal)
mean_norm_coefficients = sum_norm_coefficients / 100

print(f'Bandwidth: 10 Hz = {np.round(20 * np.pi, 1)} rad/s')
plt.plot(fs_rad, mean_norm_coefficients)
plt.xlabel('ω in radians')
plt.ylabel('Average |X(ω)|')
plt.xlim(-20*2*np.pi, 20*2*np.pi)
plt.title('Average Norm of Fourier Coefficients over 100 signals')
plt.show()

## 1.2 Gaussian power spectrum noise

**a) Time-domain signals.** Plot $x(t)$ for three randomly generated signals with `bandwidth` at $5$, $10$, and $20\,\mathrm{Hz}$. For each of these, $\mathtt{T}=1\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$ and $\mathtt{rms}=0.5$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# Modify the function so the power drops off as frequency increases
# Return both time and frequency domain signals
def generate_signal_dropoff(T, dt, rms, bandwidth, seed):
    """
    T --> The length of the signal in seconds
    dt --> Time time step in seconds
    rms --> The root mean square power of the signal
    limit --> The maximum frequency of the signal
    seed --> Random seed
    """
    ts = np.arange(0, T, dt)                                # Time points
    fs_hz = np.fft.fftshift(np.fft.fftfreq(len(ts), dt))    # Frequency bins in Hz
    fs_rad = 2 * np.pi * fs_hz                              # Frequency bins in rad/s

    # Generate half of the random signal in the frequency domain
    num_samples = len(fs_rad) // 2
    half_freq_signal = np.zeros(num_samples, dtype=np.complex128)

    for i in range(num_samples):
        stdev = np.exp(-fs_rad[i]**2 / (2 * bandwidth**2))
        real = np.random.normal(0, stdev)
        imag = np.random.normal(0, stdev)
        half_freq_signal[i] = real + 1j * imag

    # Create the full frequency signal by mirroring the half signal
    # Note: The DC component is 0, so the mean of the time domain signal is 0
    freq_signal = np.concatenate((half_freq_signal, np.array([0]), np.conj(np.flip(half_freq_signal))))

    # Cut freq_signal so it has the same length as fs (in case len(fs) is even)
    freq_signal = freq_signal[:len(fs_rad)]

    # Turn it back into the time domain
    time_signal = np.fft.ifft(np.fft.ifftshift(freq_signal))

    # Check if the time domain signal is real
    imag_threshold = 1e-10  # Error threshold
    if np.all(np.abs(time_signal.imag) < imag_threshold):
        time_signal = time_signal.real
    else:
        raise ValueError("The time domain signal is not real")

    # Scale the time and frequency signals to the RMS power
    old_rms = np.sqrt(np.mean(time_signal ** 2))
    scaling_factor = rms / old_rms
    time_signal *= scaling_factor
    freq_signal *= scaling_factor

    return ts, fs_rad, time_signal, freq_signal

# Generate and plot the signals
for bandwidth in [5, 10, 20]:
    ts, fs_rad, time_signal, freq_signal = generate_signal_dropoff(T=1, dt=0.001, rms=0.5, bandwidth=bandwidth, seed=0)
    print(f'Bandwidth: {limit} Hz = {np.round(limit * 2 * np.pi, 1)} rad/s')
    plot_signals(ts, fs_rad, time_signal, freq_signal)

**b) Average power spectrum.** Plot the average $|X(\omega)|$ (the norm of the Fourier coefficients, or “power spectrum”) over $100$ signals generated with $\mathtt{T}=1\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$, $\mathtt{rms}=0.5$, and $\mathtt{bandwidth}=10$ (of course, each of these 100 signals should have a different `seed`). The plot should have the $x$-axis labeled “$\omega$ in radians” and the average $|X|$ value for that $\omega$ on the $y$-axis.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

sum_norm_coefficients = 0
for i in range(100):
    ts, fs_rad, time_signal, freq_signal = generate_signal_dropoff(T=1, dt=0.001, rms=0.5, bandwidth=10, seed=i)
    sum_norm_coefficients += np.abs(freq_signal)
mean_norm_coefficients = sum_norm_coefficients / 100

print(f'Bandwidth: 10 Hz = {np.round(20 * np.pi, 1)} rad/s')
plt.plot(fs_rad, mean_norm_coefficients)
plt.xlabel('ω in radians')
plt.ylabel('Average |X(ω)|')
plt.xlim(-20*2*np.pi, 20*2*np.pi)
plt.title('Average Norm of Fourier Coefficients over 100 signals')
plt.show()

# 2. Simulating a spiking neuron

**a) Spike plots for constant inputs.** Plot the spike output for a constant input of $x=0$ over $1$ second. Report the number of spikes. Do the same thing for $x=1$. Use a time step of $\Delta t = 1\,\mathrm{ms}$ for the simulation.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Discussion.** Does the observed number of spikes in the previous part match the expected number of spikes for $x=0$ and $x=1$? Why or why not? What aspects of the simulation would affect this accuracy?

✍ \<YOUR SOLUTION HERE\>

**c) Spike plots for white noise inputs.** Plot the spike output for $x(t)$ generated using your function from part 1.1. Use $\mathtt{T}=1\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$, $\mathtt{rms}=0.5$, and $\mathtt{limit}=30\,\mathrm{Hz}$. Overlay on this plot $x(t)$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**d) Voltage over time.** Using the same $x(t)$ signal as in part *c)*, plot the neuron's voltage over time for the first $0.2$ seconds, along with the spikes over the same time.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) 🌟 Bonus question.** How could you improve this simulation (in terms of how closely the model matches actual equation) without significantly increasing the computation time? $0.5$ marks for having a good idea. Up to $1$ mark for actually implementing it and showing that it works.

✍ \<YOUR SOLUTION HERE\>

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# 3. Simulating two spiking neurons

**a) Spike plots for constant inputs.** Plot $x(t)$ and the spiking output for $x(t)=0$ (both neurons should spike at about $40$ spikes per second), as well as (in a separate plot) $x(t)=1$ (one neuron should spike at $\approx 150$ spikes per second, and the other should not spike at all).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Spike plots for a sinusodial input.** Plot $x(t)$ and the spiking output for $x(t)=\frac{1}2 \sin(10 \pi t)$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Spike plot for a white noise signal.** Plot $x(t)$ and the spiking output for a random signal generated with your function for question 1.1 with $\mathtt{T}=2\,\mathrm{s}$, $\mathtt{dt}=1\,\mathrm{ms}$, $\mathtt{rms}=0.5$, and $\mathtt{limit}=5\,\mathrm{Hz}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

# 4. Computing an optimal filter

**a) Document the code.** Fill in comments where there are `# !`-signs in the Python code. Make sure that your comments (where this makes sense) describe the semantics of the code and do not just repeat what is obvious from the code itself. Run the function with what you wrote for part 3 above, so that it uses the spike signal generated in 3c).

In [ ]:
def compute_optimal_filter(
        # Signal generated from your white noise generator
        x,
        # Fourier coefficients from your white noise generator
        X,
        # Spike train from the previous part
        spikes,
        # Time step size
        dt=1e-3
    ):

    # x and X should (effectively) be 1D-arrays
    assert x.ndim == 1 and X.ndim == 1
    assert x.shape[0] == X.shape[0]

    # !
    Nt = x.size

    # Make sure that "spikes" is a 2 x Nt array
    assert spikes.ndim == 2
    assert spikes.shape[0] == 2              
    assert spikes.shape[1] == Nt

    # !
    T = Nt * dt

    # !
    ts = np.arange(Nt) * dt - T / 2.0

    # !
    fs = np.arange(Nt) / T - Nt / (2.0 * T)

    # !
    omega = fs * 2.0 * np.pi

    # !
    r = spikes[0] - spikes[1]

    # !
    R = np.fft.fftshift(np.fft.fft(r))

    # !
    sigma_t = 25e-3

    # !
    W2 = np.exp(-omega**2*sigma_t**2)

    # !
    W2 = W2 / sum(W2)

    # !
    CP = X*R.conjugate()

    # !
    WCP = np.convolve(CP, W2, 'same')

    # !
    RP = R*R.conjugate()

    # !
    WRP = np.convolve(RP, W2, 'same')

    # !
    XP = X*X.conjugate()

    # !
    WXP = np.convolve(XP, W2, 'same')

    # !
    H = WCP / WRP

    # !
    h = np.fft.fftshift(np.fft.ifft(np.fft.ifftshift(H))).real

    # !
    XHAT = H*R

    # !
    xhat = np.fft.ifft(np.fft.ifftshift(XHAT)).real

    return ts, fs, R, H, h, XHAT, xhat, XP, WXP

**b) Optimal filter.** Plot the time and frequency plots of the optimal filter for the signal you generated in question 3c). Make sure to use appropriate limits for the $x$-axis.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**c) Decoded signal.** Plot the $x(t)$ signal, the spikes, and the decoded $\hat x(t)$ value for the signal from 3c).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**d) Power spectra.** Plot the signal $|X(\omega)|$, spike response $|R(\omega)|$, and filtered signal $|\hat X(\omega)|$ power spectra for the signal from 3c).

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**e) Discussion.** How do these spectra relate to the optimal filter?

✍ \<YOUR SOLUTION HERE\>

**f) Filter for different signal bandwidths.** Plot the optmial filter $h(t)$ in the time domain when filtering spike trains for white noise signals with different `limit` values of $2\,\mathrm{Hz}$, $10\,\mathrm{Hz}$, and $30\,\mathrm{Hz}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**g) Discussion.** Describe the effects on the time plot of the optimal filter as `limit` increases. Why does this happen?

✍ \<YOUR SOLUTION HERE\>

# Using post-synaptic currents as a filter


**a) Plotting the filter for different $n$.** Plot the normalized $h(t)$ for $n=0$, $1$, and $2$, with $\tau=7\,\mathrm{ms}$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**b) Discussion.** What two things do you expect increasing $n$ will do to $\hat{x}(t)$?

✍ \<YOUR SOLUTION HERE\>

**c) Plotting the filter for different $\tau$.** Plot the normalized $h(t)$ for $\tau=2\,\mathrm{ms}$, $\tau=5\,\mathrm{ms}$, $\tau=10\,\mathrm{ms}$, $\tau=20\,\mathrm{ms}$ with $n = 0$.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**d) Discussion.** What two things do you expect increasing $\tau$ will do to $\hat{x}(t)$?

✍ \<YOUR SOLUTION HERE\>

**e) Decoding a spike-train using the post-synaptic current filter.** Decode $\hat{x}(t)$ from the spikes generated in question 3c) using an $h(t)$ with $n=0$ and $\tau=7\,\mathrm{ms}$. Do this by generating the spikes, filtering them with $h(t)$, and using that as your activity matrix $A$ to compute your decoders. Plot the time and frequency plots for this $h(t)$. Plot the $x(t)$ signal, the spikes, and the decoded $\hat{x}(t)$ value.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**f) Deocding a spike-train representing a low-frequency signal.** Use the same decoder and $h(t)$ as in part e), but generate a new $x(t)$ with $\mathtt{limit}=2\,\mathrm{Hz}$. Plot the $x(t)$ signal, the spikes, and the decoded $\hat{x}(t)$ value.

In [ ]:
# ✍ <YOUR SOLUTION HERE>

**g) Discussion.** How do the decodings from e) and f) compare? Explain.

✍ \<YOUR SOLUTION HERE\>